In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.current_device()

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
torch.cuda.memory_reserved()

In [ ]:
# to use gpu in pytorch, first we should set our tensors in gpu

In [ ]:
a = torch.FloatTensor([1.0, 2.0])
a.device

In [ ]:
a = torch.FloatTensor([1.0, 2.0]).cuda()
a.device

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
# to use gpu in pytorch, second we should set our model in gpu

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, out_features)
        
    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return X

In [ ]:
torch.manual_seed(32)
model = Model()

In [ ]:
next(model.parameters()).is_cuda

In [ ]:
gpu_model = model.cuda()

In [ ]:
next(model.parameters()).is_cuda

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
# let's have an example

In [ ]:
df = pd.read_csv('../data/ann_files/iris.csv')

In [ ]:
X = df.drop('target', axis=1).values
y = df['target'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [ ]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [ ]:
# IF we want to define DATALOADER, we should set it for CUDA as following:
# trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
# testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
start_time = time.time()

epochs = 100
losses = []

for i in range(epochs):
    y_pred = gpu_model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    if i%10 == 0:
        print(f'Epoch:{i}, loss:{loss.item()}')
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


total_time = time.time() - start_time
print(f'Duration: {total_time} seconds')

In [ ]:
for i, data in enumerate(X_test):
    print(i, data)

In [ ]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = gpu_model.forward(data)
        print(f'{i+1:2}. {str(y_val):38} {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f'{correct} out of {len(y_test)} = {correct*100/len(y_test)}%')